<a href="https://colab.research.google.com/github/MarkStephens060482/MarkStephens060482/blob/main/Panel_Data_Regression_Climate_Change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Panel Data Regression
### Action on climate change

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd, seaborn as sns, numpy as np,  matplotlib.pyplot as plt, datetime as dt, scipy.stats as st
from pprint import pprint
import os, csv

path = '/content/gdrive/MyDrive/Colab Notebooks/'

try:
  cwd = os.chdir(path)
   
  print("Current working directory: {0}".format(os.getcwd()))
    
except FileNotFoundError:
    print("Directory: {0} does not exist".format(path))
except NotADirectoryError:
    print("{0} is not a directory".format(path))
except PermissionError:
    print("You do not have permissions to change to {0}".format(path))

Current working directory: /content/gdrive/MyDrive/Colab Notebooks


In [ ]:
!pip install scikit-lego
!pip install linearmodels
!pip install miceforest

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklego.model_selection import GroupTimeSeriesSplit
from linearmodels import PanelOLS, RandomEffects
from linearmodels.panel import PooledOLS
import statsmodels.api as sm
import miceforest as mf

categories = ['Climate Change Performance Index',
                           'Corruption Perception Index',
                           'Democracy Index',
                           'Education Index',
                           'Income Index',
                           'Inequality-adjusted Education Index',
                           'Inequality adjusted Income Index',
                           'Global Peace Index',
                           'Environmental Performance Index',
                           'Human Development Index']


file_name = "data_set_impute_ready2.csv"
with open('/content/gdrive/MyDrive/Colab Notebooks/data_set_impute_ready2.csv',encoding="utf-8") as dataFile:
  df = pd.read_csv(dataFile,low_memory=False)
          

In [ ]:
df['country'] = pd.Categorical(df['country'])
df['Year'] = pd.Categorical(df['Year'])

In [ ]:
imputer = mf.ImputationKernel(df,
                              datasets = 20,
                              categorical_feature="auto",
                              random_state = 21)

imputer.mice(20)


In [ ]:
imputer.plot_correlations(wspace=0.4,hspace=0.5)

NameError: ignored

# Effects  hypothesis Tests
### Determine Your Model

In order to determine your model, conduct necessary tests: F test for the fixed effect model and Breusch-Pagan Lagrange Multiplier (LM) test. Hausman test is needed if you find both fixed and random effects. See the next section for the test. Once you finish F and LM tests, determine your model as shown in the following table.

| Fixed effect (F test or Wald test) Performed on Fixed effects model| Random effect (Breusch-Pagan LM test) Performed on OLS model | Correct model |
| :---------------------------------| :-------------------------------------|:-------------:|
| **H0 is not rejected** (No fixed effect)| **H0 is not rejected** (No random effect) | Data are poolable **Pooled OLS**|
|  | | |
|**H0 is rejected** (fixed effect)| **H0 is not rejected**  (No random effect)   | **Fixed effect model**|
|  || |
| **H0 is not rejected** (No fixed effect) | **H0 is rejected** (random effect)| **Random effect model**|
| | | |
| **H0 is rejected** (fixed effect) | **H0 is rejected** (random effect) | (1) Fixed and random effect model or (2) choose one of the two depending on the result of Hausman test (recommended direction).
 |
|  | | |




In [ ]:
from scipy.stats import chi2
def FERE_test(model_fit):
    # model tests for FE and RE
    if hasattr(model_fit, 'name'):
        if model_fit.name == 'PooledOLS':
            # Breusch and Pagan (1980) Langrange - Multiplier for random effects based on OLS residuals.
            # entity specific means of OLS residuals
            e_bar = model_fit.resids.groupby('country').mean()
            e = model_fit.resids
            T = model_fit.entity_info[0]
            n = model_fit.entity_info[4]
            #LM statistics 
            LM = ((n*T)/(2*(T-1)))*((((T**2)*e_bar.T.dot(e_bar))/(e.T.dot(e))) - 1)**2
            # Determine p-value
            p_value = 1 - chi2.cdf(LM, 1)
            if p_value < 0.05:
                return "Random effects"
            else:
                return "No random effects"

        elif model_fit.name == 'PanelOLS':
            # Perform an F-test  for Pooling data on Fixed effects model
            FE_test = model_fit.f_pooled.pval
            if FE_test < 0.05:
                return "Fixed effects"
            else:
                return "No fixed effects"
        else:
            return "no test"
    else:
        return "no test"
        

In [ ]:
from sklearn.dummy import DummyRegressor

predict_dict = {}
FE_RE_tests_dict={}

for i in range(1,21):
  dataset = imputer.complete_data(i-1)  
  # drop variables from dataframe.
  climate_df = dataset.drop(columns = ['education_index',
                                       'income_index',
                                       'ia_income_index',
                                       'gpi'])
  
  #Panel Regression models
  
  # set indexes to develop panel data structure
  year = pd.Categorical(climate_df.Year)
  country = pd.Categorical(climate_df.country)
  
  # feature matrix and target variable for training set
  X_train = climate_df[climate_df['Year'] < 2020].drop(columns = ['ccpi'])
  
  # Set indices
  X_train = X_train.set_index(['country','Year'])
  
  # target variable
  y_train = climate_df.loc[climate_df['Year'] < 2020,['ccpi','Year','country']]
  
    # Set indices
  y_train = y_train.set_index(['country','Year'])
  
  # feature matrix and target variable for testing set
  X_test = climate_df[climate_df['Year'] >= 2020].drop(columns = ['ccpi'])
  
  # Set indices
  X_test = X_test.set_index(['country','Year'])
  
  #make country and Year an index
  y_test = climate_df.loc[climate_df['Year'] >= 2020,['ccpi','Year','country']]
  
  # Set indices
  y_test = y_test.set_index(['country','Year'])
  
  #sklearn
  #Dummy Regressor
  dummy = DummyRegressor()
  modeldummy = dummy.fit(X_train, y_train)
  
  # Pooled Ordinary Least Squares model
  exog = sm.add_constant(X_train)
  
  #Linear Models    
  #Pooled Model
  model1 = PooledOLS(y_train, exog)
  # Fixed effects in countries - time invariant
  model2 = PanelOLS(y_train, exog, entity_effects=True)
  # Fixed effects in time - individual invariant
  model3 = PanelOLS(y_train, exog, time_effects= True)
  # Mixed effects in countries and years/
  model4 = PanelOLS(y_train, exog, entity_effects=True, time_effects= True)
  # Random Effects Model
  model5 = RandomEffects(y_train, exog)
  
  models = [model1,model2,model3,model4,model5]
  
  # loop through Panel Regression models and fit data, collect parameter estimates     
  for j, model in enumerate(models):
      
      model_fit = model.fit(cov_type="clustered", cluster_entity=True)
          
      y_pred = model_fit.predict(sm.add_constant(X_test)).rename(columns={"predictions": "ccpi"+str(i+1)})
      
      if len(predict_dict) < len(models):
          # model predictions for first imputation
          predict_dict[str(model_fit.name)+str(j+1)] = [y_pred]
          # Model tests and evaluations  based on OLS and Fixed effects model for first imputation
          FE_RE_tests_dict[str(model_fit.name)+str(j+1)] = [FERE_test(model_fit)]
          continue
                    
      # model predictions for second to last imputation
      predict_dict[str(model_fit.name)+str(j+1)].append(y_pred)
      # Model tests and evaluations based on OLS and Fixed effects model for second to last imputation
      FE_RE_tests_dict[str(model_fit.name)+str(j+1)].append(FERE_test(model_fit))
  
  #prediction on Dummy Regressor
  y_pred_dummy = pd.DataFrame(modeldummy.predict(X_test),columns = y_pred.columns, index = y_pred.index)
  
  #add predictions on dummy regression to model predictions dictionary for first imputation
  if len(predict_dict) < len(models)+1:
      predict_dict['dummy'] = [y_pred_dummy]
      FE_RE_tests_dict['dummy']=[FERE_test(modeldummy.fit(X_train,y_train))]
      continue
  
  #add predictions on dummy regression to model predictions dictionary for second to last imputation
  predict_dict['dummy'].append(y_pred_dummy)
  FE_RE_tests_dict['dummy'].append(FERE_test(modeldummy.fit(X_train,y_train)))
  

# Rubin's Rules - Combining estimates from model of imputation data
For a single population parameter of interest, Q, e.g. a regression coefficient, the MI overall point estimate is the average of the m estimates of Q from the imputed datasets: 
1.  Q_bar = (1/m).sum(Q_hat_i) 

The associated total variance for this overall MI estimate is: 
1. T = U_hat + ( 1 + 1/m)B, where
2. U_hat = (1/m).sum(U_i) is the estimated within imputation variance.
3. B = (1/(m-1)).sum(Q_hat - Q_bar)^2 



In [ ]:
# Combine point estimates using Rubin's Rules and evaluate performance 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from statistics import mode 

mean_predict_dict = {}
withinvar_predict_dict = {}
betweenvar_predict_dict = {}
RMSE_dict = {}
r2_dict = {}
mode_REFE_test_outcome_dict = {}

for key in predict_dict.keys():
    # Q_bar per model
    mean_predict_dict[key] = pd.concat(predict_dict[key], axis = 1).mean(axis =1)
    
    # Determine mode of Fixed Effect and Random Effect Test outcomes for each model.
    mode_REFE_test_outcome_dict[key] = mode(FE_RE_tests_dict[key])
    
    # between Imputation Variance, B
    betweenvar_predict_dict[key] = (1/(n-1))*(pd.concat(predict_dict[key], axis = 1).sub(mean_predict_dict[key], axis = 0))**2  
    
    # Standard Error of the prediction
    #sterr_pred_dict[key] = 
    
    # Within Imputation Variance
    #withinvar_predict_dict[key] = pd.concat(sterr_pred_dict[key], axis = 1).mean(axis =1)
    
    
    # Root Mean Squared Error
    RMSE_dict[key] = round(mean_squared_error(y_test, mean_predict_dict[key],squared=False),1)
    
    
    # R squared score
    r2_dict[key] = round(100*r2_score(y_test, mean_predict_dict[key]),1)
    
# Present Table of model evaluates
pd.DataFrame([RMSE_dict,r2_dict,mode_REFE_test_outcome_dict],columns =list(mean_predict_dict.keys()) ,index = ["RMSE",'r2','Test outcome'])

    


,PooledOLS1,PanelOLS2,PanelOLS3,PanelOLS4,RandomEffects5,dummy
RMSE,11.9,13.3,12.0,13.4,12.1,13.7
r2,15.5,-6.4,12.7,-7.5,11.7,-12.6
Test outcome,Random effects,Fixed effects,Fixed effects,Fixed effects,no test,no test


In [ ]:
from linearmodels.panel.results import compare
table = {
    "Pooled" : model1.fit(cov_type="clustered", cluster_entity=True),
    "Entity FE" : model2.fit(cov_type="clustered", cluster_entity=True),
    "Time FE" :  model3.fit(cov_type="clustered", cluster_entity=True),
    "Mixed Effects" : model4.fit(cov_type="clustered", cluster_entity=True),
    "Random Effects" : model5.fit(cov_type="clustered", cluster_entity=True)}
    
compare(table, stars = True)

,Pooled,Entity FE,Time FE,Mixed Effects,Random Effects
Dep. Variable,ccpi,ccpi,ccpi,ccpi,ccpi
Estimator,PooledOLS,PanelOLS,PanelOLS,PanelOLS,RandomEffects
No. Observations,660,660,660,660,660
Cov. Est.,Clustered,Clustered,Clustered,Clustered,Clustered
R-squared,0.2971,0.0239,0.3580,0.0334,0.0679
R-Squared (Within),-0.0729,0.0239,-0.4474,0.0031,0.0151
R-Squared (Between),0.4438,0.0086,0.5147,-0.3989,0.3585
R-Squared (Overall),0.2971,0.0130,0.2415,-0.2847,0.2609
F-statistic,55.288,2.9380,71.710,4.0762,9.5230
P-value (F-stat),0.0000,0.0125,0.0000,0.0012,0.0000
